Installing Libraries

In [ ]:
!pip install PyDrive

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download = drive.CreateFile({'id': '1BDNB1jirJqrfSbst5dMInbVNhmmtEx-F'})

In [ ]:
download.GetContentFile('dataset.tar')

In [ ]:
!tar xvf dataset.tar

Streaming output truncated to the last 5000 lines.
archive/ASL/A/A_1813.jpg
archive/ASL/A/A_898.jpg
archive/ASL/A/A_429.jpg
archive/ASL/A/A_92.jpg
archive/ASL/A/A_1369.jpg
archive/ASL/A/A_529.jpg
archive/ASL/A/A_799.jpg
archive/ASL/A/A_1469.jpg
archive/ASL/A/A_2494.jpg
archive/ASL/A/A_2365.jpg
archive/ASL/A/A_694.jpg
archive/ASL/A/A_2851.jpg
archive/ASL/A/A_1963.jpg
archive/ASL/A/A_1107.jpg
archive/ASL/A/A_1929.jpg
archive/ASL/A/A_3860.jpg
archive/ASL/A/A_1481.jpg
archive/ASL/A/A_2327.jpg
archive/ASL/A/A_1864.jpg
archive/ASL/A/A_3493.jpg
archive/ASL/A/A_2268.jpg
archive/ASL/A/A_3985.jpg
archive/ASL/A/A_3707.jpg
archive/ASL/A/A_3096.jpg
archive/ASL/A/A_1995.jpg
archive/ASL/A/A_2744.jpg
archive/ASL/A/A_3346.jpg
archive/ASL/A/A_4020.jpg
archive/ASL/A/A_754.jpg
archive/ASL/A/A_3850.jpg
archive/ASL/A/A_2032.jpg
archive/ASL/A/A_1149.jpg
archive/ASL/A/A_3548.jpg
archive/ASL/A/A_2007.jpg
archive/ASL/A/A_456.jpg
archive/ASL/A/A_3620.jpg
archive/ASL/A/A_1430.jpg
archive/ASL/A/A_1671.jpg
archive/

Classes

In [ ]:
training_dir="archive/ASL"
content=sorted(os.listdir(training_dir))
print(content)
len(content)

['1', '3', '4', '5', '7', '8', '9', 'A', 'B', 'Baby', 'Brother', 'C', 'D', 'Dont_like', 'E', 'F', 'Friend', 'G', 'H', 'Help', 'House', 'I', 'J', 'K', 'L', 'Like', 'Love', 'M', 'Make', 'More', 'N', 'Name', 'No', 'O_OR_0', 'P', 'Pay', 'Play', 'Q', 'R', 'S', 'Stop', 'T', 'U', 'V_OR_2', 'W_OR_6', 'With', 'X', 'Y', 'Yes', 'Z', 'nothing']


51

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.models import Sequential,Model
from keras.layers import Dense,Flatten,Dropout,BatchNormalization,Conv2D,MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from  skimage.transform import resize
from keras.utils import to_categorical
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
import pickle

In [ ]:
data_generator = ImageDataGenerator(
    samplewise_center=True, 
    samplewise_std_normalization=True,
    brightness_range=[0.8, 1.0],
    zoom_range=[1.0, 1.2],
    validation_split=0.1
)

train_generator = data_generator.flow_from_directory(training_dir, target_size=(200,200), shuffle=True, seed=13,
                                                     class_mode='categorical', batch_size=64, subset="training")

validation_generator = data_generator.flow_from_directory(training_dir, target_size=(200, 200), shuffle=True, seed=13,
                                                     class_mode='categorical', batch_size=64, subset="validation")


Found 182700 images belonging to 51 classes.
Found 20300 images belonging to 51 classes.


CNN Model

In [ ]:
def create_model():
    
    model = Sequential()
    
    model.add(Conv2D(64, kernel_size = [3,3], padding = 'same', activation = 'relu', input_shape = (200,200,3)))
    model.add(MaxPool2D(pool_size = [3,3]))
    
    model.add(Conv2D(128, kernel_size = [5,5], padding = 'same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = [3,3]))
    
    model.add(Conv2D(256, kernel_size = [3,3], padding = 'same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = [3,3]))
    
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.5))
    
    model.add(Dense(1024, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
    model.add(Dense(512, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
    model.add(Dense(51, activation = 'softmax'))
    
    print("MODEL CREATED")
    return model

In [ ]:
from keras import regularizers
model = create_model()
model.summary()

MODEL CREATED
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 200, 200, 64)      1792      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 66, 66, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 66, 66, 128)       204928    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 22, 22, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 256)       295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 256)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7

In [ ]:
model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ["accuracy"])

In [ ]:
model_hist = model.fit_generator(train_generator,
                                validation_data=validation_generator,
                                steps_per_epoch=200,
                                validation_steps=50,
                                epochs=5,
                              )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
200/200 [==============================] - 2236s 11s/step - loss: 4.8298 - accuracy: 0.3573 - val_loss: 4.2439 - val_accuracy: 0.3638
Epoch 2/5
200/200 [==============================] - 2244s 11s/step - loss: 3.2177 - accuracy: 0.6063 - val_loss: 3.4699 - val_accuracy: 0.4969
Epoch 3/5
200/200 [==============================] - 2241s 11s/step - loss: 2.6312 - accuracy: 0.7157 - val_loss: 3.6013 - val_accuracy: 0.4800
Epoch 4/5
200/200 [==============================] - 2235s 11s/step - loss: 2.3704 - accuracy: 0.7604 - val_loss: 2.8603 - val_accuracy: 0.6006
Epoch 5/5
200/200 [==============================] - 2245s 11s/step - loss: 2.1744 - accuracy: 0.7925 - val_loss: 2.7417 - val_accuracy: 0.6484


In [ ]:
import tensorflow as tf

In [ ]:
# saving the model in tensorflow format
model.save('./MyModel_tf',save_format='tf')


# loading the saved model
loaded_model = tf.keras.models.load_model('./MyModel_tf')

INFO:tensorflow:Assets written to: ./MyModel_tf/assets


In [ ]:
# retraining the model
loaded_model.fit_generator(train_generator,
                                validation_data=validation_generator,
                                steps_per_epoch=200,
                                validation_steps=50,
                                epochs=5,
                              )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
200/200 [==============================] - 2232s 11s/step - loss: 2.0827 - accuracy: 0.7930 - val_loss: 2.5592 - val_accuracy: 0.6812
Epoch 2/5
200/200 [==============================] - 2231s 11s/step - loss: 2.0623 - accuracy: 0.8213 - val_loss: 2.6603 - val_accuracy: 0.6881
Epoch 3/5
200/200 [==============================] - 2220s 11s/step - loss: 1.9798 - accuracy: 0.8330 - val_loss: 2.4316 - val_accuracy: 0.7169
Epoch 4/5
200/200 [==============================] - 2224s 11s/step - loss: 1.8689 - accuracy: 0.8486 - val_loss: 2.5982 - val_accuracy: 0.6709
Epoch 5/5
200/200 [==============================] - 2230s 11s/step - loss: 1.7778 - accuracy: 0.8609 - val_loss: 2.2057 - val_accuracy: 0.7294


In [ ]:
# saving the model in tensorflow format
loaded_model.save('./MyModel_1_tf',save_format='tf')


# loading the saved model
loaded_model_1 = tf.keras.models.load_model('./MyModel_1_tf')

INFO:tensorflow:Assets written to: ./MyModel_1_tf/assets


In [ ]:
# retraining the model
loaded_model_1.fit_generator(train_generator,
                                validation_data=validation_generator,
                                steps_per_epoch=200,
                                validation_steps=50,
                                epochs=5
                              )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
200/200 [==============================] - 2249s 11s/step - loss: 1.6461 - accuracy: 0.8597 - val_loss: 2.0712 - val_accuracy: 0.7297
Epoch 2/5
200/200 [==============================] - 2250s 11s/step - loss: 1.6266 - accuracy: 0.8655 - val_loss: 2.2256 - val_accuracy: 0.7416
Epoch 3/5
200/200 [==============================] - 2248s 11s/step - loss: 1.6440 - accuracy: 0.8771 - val_loss: 2.1682 - val_accuracy: 0.7394
Epoch 4/5
200/200 [==============================] - 2247s 11s/step - loss: 1.5986 - accuracy: 0.8826 - val_loss: 1.9459 - val_accuracy: 0.7803
Epoch 5/5
200/200 [==============================] - 2254s 11s/step - loss: 1.4755 - accuracy: 0.8931 - val_loss: 1.8824 - val_accuracy: 0.7834


After training the model 3 times:

 Training Accuracy = 89.31%

 Testing Accuracy = 78.34%

In [ ]:
t = 2249+2250+2248+2247+2254
print('Total training time:',t,' sec')

Total training time: 11248  sec
